In [18]:
#!python -m pip install pyserini --user
#!python -m pip install jsonlines --user

In [1]:
!ls -l --block-size=M /mnt/data/factcheck/fever/data-en-latest/

total 26835M
drwxr-xr-x   5 drchajan k13136        1M Sep  9 23:17 emb
-rw-r--r--   1 drchajan factcheck  5972M Aug  3 18:20 enwiki.jsonl
-rw-r--r--   1 drchajan factcheck 14103M Aug  3 17:12 enwiki.xml
drwxr-xr-x 144 drchajan factcheck     1M Jul 29 22:49 extracted
drwxr-xr-x   2 drchajan factcheck     1M Aug 12 17:04 fever
drwxr-xr-x   2 drchajan factcheck     1M Aug  5 12:16 fever-data
drwxr-xr-x   2 drchajan factcheck     1M Aug  5 09:45 index
-rw-r--r--   1 drchajan factcheck     1M Aug  5 20:44 info.txt
drwxr-xr-x   2 drchajan k13136        1M Sep  9 22:38 predictions
drwxr-xr-x   2 drchajan factcheck     0M Aug  6 12:09 wiki-pages


In [2]:
!cat /mnt/data/factcheck/fever/data-en-latest/info.txt

This is EN Wiki dataset based on latest Wiki snapshot 
 - abstracts only (as in original)
 - PTB tokenization using NLTK + fixes (mainly bracket tockenization) using Jupyter notebook (TODO: add to main code-base)
 - note that some pages were renamed since original Fever Wiki snapshot
 
Updates:
05/08/2020 - fixed titles to match original Fever tokenization



# Run on FEVER data

In [6]:
DATAFOLDER='/mnt/data/factcheck/fever/data-en-latest/'

In [7]:
!ls /mnt/data/factcheck/fever/data-en-latest

emb	      enwiki.xml  fever       index	predictions
enwiki.jsonl  extracted   fever-data  info.txt	wiki-pages


### Prepare the wiki dump into format suitable for anserini

In [37]:
!python /home/ryparmar/pyserini/src/convert_collection_to_jsonl.py \
--collection_folder /mnt/data/factcheck/fever/data-en-latest/ \
--output_folder /home/ryparmar/pyserini/en-latest/data \
--max_docs_per_file 10000000 \
--granularity 'paragraph'

Converting collection...
Converted 100000 docs in 1 files
Converted 200000 docs in 1 files
Converted 300000 docs in 1 files
Converted 400000 docs in 1 files
Converted 500000 docs in 1 files
Converted 600000 docs in 1 files
Converted 700000 docs in 1 files
Converted 800000 docs in 1 files
Converted 900000 docs in 1 files
Converted 1000000 docs in 1 files
Converted 1100000 docs in 1 files
Converted 1200000 docs in 1 files
Converted 1300000 docs in 1 files
Converted 1400000 docs in 1 files
Converted 1500000 docs in 1 files
Converted 1600000 docs in 1 files
Converted 1700000 docs in 1 files
Converted 1800000 docs in 1 files
Converted 1900000 docs in 1 files
Converted 2000000 docs in 1 files
Converted 2100000 docs in 1 files
Converted 2200000 docs in 1 files
Converted 2300000 docs in 1 files
Converted 2400000 docs in 1 files
Converted 2500000 docs in 1 files
Converted 2600000 docs in 1 files
Converted 2700000 docs in 1 files
Converted 2800000 docs in 1 files
Converted 2900000 docs in 1 file

In [22]:
!ls -l --block-size=M /home/ryparmar/pyserini/data

total 8101M
-rw-r--r-- 1 ryparmar k13136 2981M Sep  8 13:27 docs00-id=int.json
-rw-r--r-- 1 ryparmar k13136 3078M Sep  8 19:53 docs00.json


In [25]:
!head -n 1 /mnt/data/factcheck/fever/data-en-latest/enwiki.jsonl > orig  #| sed "s/ /\n/g"
!head -n 1 /home/ryparmar/pyserini/data/docs00.json > parsed
!diff orig parsed

1c1
< {"id": "Sláintecare", "text": "Sláintecare is a proposed reform of the healthcare system of Ireland . Under the existing health service system large out-of-pocket payments are required ; the Republic of Ireland is the only state within the EU which does not provide universal healthcare , according to WHO criteria . The newly elected Irish government committed itself in March 2011 to reform the two-tier structure of national health service in response to public discontent with austerity measures from 2008 onwards . Since public funding of health provision was reduced between 2009 and 2013 to meet payment obligations arising from the state 's budget deficit , citizens faced higher medical charges and extended waiting lists . In reaction to this development , the coalition government of Fine Gael and the Labour Party declared to introduce '' a universal single-tiered health service , which guarantees access based on need , not income ... through Universal Health Insurance. ” The pub

In [26]:
"Sláintecare" == "Sl\u00e1intecare"

True

In [28]:
import jsonlines
with jsonlines.open('/mnt/data/factcheck/fever/data-en-latest/enwiki.jsonl') as r:
    count = 0
    for i in r:
        print(i)
        count += 1
        if count == 1:
            break

{'id': 'Sláintecare', 'text': "Sláintecare is a proposed reform of the healthcare system of Ireland . Under the existing health service system large out-of-pocket payments are required ; the Republic of Ireland is the only state within the EU which does not provide universal healthcare , according to WHO criteria . The newly elected Irish government committed itself in March 2011 to reform the two-tier structure of national health service in response to public discontent with austerity measures from 2008 onwards . Since public funding of health provision was reduced between 2009 and 2013 to meet payment obligations arising from the state 's budget deficit , citizens faced higher medical charges and extended waiting lists . In reaction to this development , the coalition government of Fine Gael and the Labour Party declared to introduce '' a universal single-tiered health service , which guarantees access based on need , not income ... through Universal Health Insurance. ” The published

In [30]:
import jsonlines
with jsonlines.open('/home/ryparmar/pyserini/data/docs00.json') as r:
    count = 0
    for i in r:
        print(i)
        count += 1
        if count == 1:
            break

{'id': 'Sláintecare', 'contents': "Sláintecare is a proposed reform of the healthcare system of Ireland . Under the existing health service system large out-of-pocket payments are required ; the Republic of Ireland is the only state within the EU which does not provide universal healthcare , according to WHO criteria . The newly elected Irish government committed itself in March 2011 to reform the two-tier structure of national health service in response to public discontent with austerity measures from 2008 onwards . Since public funding of health provision was reduced between 2009 and 2013 to meet payment obligations arising from the state 's budget deficit , citizens faced higher medical charges and extended waiting lists . In reaction to this development , the coalition government of Fine Gael and the Labour Party declared to introduce '' a universal single-tiered health service , which guarantees access based on need , not income ... through Universal Health Insurance. ” The publi

### Try to compute index

In [31]:
!python -m pyserini.index \
-collection JsonCollection \
-generator DefaultLuceneDocumentGenerator \
-threads 1 \
-input /home/ryparmar/pyserini/en-latest/data \
-index /home/ryparmar/pyserini/en-latest/indexes-full \
-storePositions -storeDocvectors -storeRaw

2020-09-08 19:59:19,862 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to INFO
2020-09-08 19:59:19,865 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Starting indexer...
2020-09-08 19:59:19,865 INFO  [main] index.IndexCollection (IndexCollection.java:640) - ============ Loading Parameters ============
2020-09-08 19:59:19,865 INFO  [main] index.IndexCollection (IndexCollection.java:641) - DocumentCollection path: /home/ryparmar/pyserini/data
2020-09-08 19:59:19,866 INFO  [main] index.IndexCollection (IndexCollection.java:642) - CollectionClass: JsonCollection
2020-09-08 19:59:19,866 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Generator: DefaultLuceneDocumentGenerator
2020-09-08 19:59:19,867 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Threads: 1
2020-09-08 19:59:19,867 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Stemmer: porter
2020-09-08 19:59:19,867 INFO  [main] index.IndexCo

In [32]:
!ls /home/ryparmar/pyserini/indexes-full

_0.fdt	_0.nvm	_0_Lucene50_0.doc  _0_Lucene80_0.dvd
_0.fdx	_0.si	_0_Lucene50_0.pos  _0_Lucene80_0.dvm
_0.fnm	_0.tvd	_0_Lucene50_0.tim  segments_1
_0.nvd	_0.tvx	_0_Lucene50_0.tip  write.lock


### Test the computed index

In [133]:
from pyserini import analysis, index
from pyserini.search import SimpleSearcher

INDEXER = '/home/ryparmar/pyserini/indexes-full'
# index_reader = index.IndexReader(INDEXER)

searcher = SimpleSearcher(INDEXER)

query = 'Sláintecare should be the name of healthcare system in Ireland'
hits = searcher.search(query)

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 0               16.15490
 2 3655984         10.95880
 3 1736638         10.29230
 4 1949761         10.22800
 5 5495217         9.64750
 6 1580268         9.35750
 7 1390315         9.22520
 8 3014088         9.14190
 9 1354987         9.14100
10 2651102         9.12890


### Check the computed index and compare with the one provided by DrQA -- MOVE TO THE END!

In [166]:
import itertools
for term in itertools.islice(index_reader.terms(), 1000):
    print(f'{term.term} (df={term.df}, cf={term.cf})')

0 (df=4, cf=6)
0.6 (df=1, cf=1)
08 (df=2, cf=2)
1 (df=39, cf=45)
1,139 (df=1, cf=1)
1,345 (df=1, cf=1)
1,500 (df=1, cf=1)
1,515 (df=1, cf=1)
1,600 (df=1, cf=1)
1,775 (df=1, cf=1)
1,800 (df=1, cf=1)
1.4 (df=1, cf=1)
1.5 (df=1, cf=1)
1.80 (df=1, cf=1)
10 (df=34, cf=46)
10,000 (df=3, cf=3)
100 (df=10, cf=15)
1000 (df=1, cf=1)
103 (df=1, cf=1)
1034 (df=1, cf=1)
1041 (df=1, cf=1)
1054 (df=1, cf=1)
10k (df=1, cf=1)
10th (df=2, cf=2)
11 (df=30, cf=31)
115,000 (df=1, cf=1)
1164 (df=1, cf=1)
1165 (df=1, cf=1)
117 (df=1, cf=1)
1170 (df=1, cf=1)
118 (df=1, cf=1)
1180 (df=1, cf=1)
11th (df=3, cf=3)
12 (df=27, cf=30)
12,000 (df=1, cf=1)
120 (df=3, cf=3)
1200 (df=1, cf=1)
121 (df=1, cf=1)
122 (df=1, cf=1)
1240 (df=1, cf=1)
1245 (df=1, cf=1)
1247 (df=1, cf=1)
1253 (df=1, cf=1)
129 (df=1, cf=1)
12th (df=5, cf=5)
13 (df=31, cf=33)
1316 (df=1, cf=1)
1324 (df=1, cf=1)
1325 (df=1, cf=1)
132nd (df=1, cf=1)
1332 (df=1, cf=1)
1371 (df=1, cf=1)
1381 (df=1, cf=1)
1384 (df=1, cf=1)
1395 (df=1, cf=1)
13th (df=6,

In [163]:
term = 'cities'

# Look up its document frequency (df) and collection frequency (cf).
# Note, we use the unanalyzed form:
df, cf = index_reader.get_term_counts(term)
print(f'term "{term}": df={df}, cf={cf}')

term "cities": df=41, cf=51


In [164]:
doc_vector = index_reader.get_document_vector('0')
print(doc_vector)

{'govern': 5, 'hospit': 2, 'nation': 2, 'year': 1, 'envisag': 1, 'commit': 1, 'led': 1, 'academ': 1, 'parliament': 1, 'cover': 3, 'path': 1, 'would': 4, 'róisín': 1, 'draft': 1, 'annual': 1, 'approxim': 2, 'state': 3, 'ten': 1, 'addition': 1, '11': 1, 'ag': 1, 'gener': 1, 'statu': 1, 'cours': 1, 'extend': 1, 'uhc': 1, '3': 1, 'highest': 1, 'foresaw': 1, 'declar': 1, 'coalit': 1, 'access': 3, 'dublin': 1, 'debat': 1, 'document': 2, 'healthcar': 7, 'turn': 1, 'two': 1, 'elect': 2, 'kei': 1, 'polici': 1, 'forese': 1, 'democrat': 1, 'found': 2, 'same': 1, 'larg': 1, 'measur': 1, 'after': 2, 'free': 2, '30': 1, 'parliamentarian': 1, 'reaction': 1, 'expenditur': 1, 'right': 1, 'team': 1, 'acut': 1, 'sláintecar': 2, 'march': 1, 'call': 1, 'td': 1, 'charg': 3, 'face': 1, 'june': 1, 's': 1, 'abandon': 1, 'infrastructur': 1, 'meet': 1, 'appoint': 1, 'under': 2, 'through': 1, 'definit': 1, 'featur': 1, 'offic': 1, 'invest': 1, 'project': 1, 'parti': 2, 'shortal': 1, 'outlin': 1, 'structur': 1, 'w

## Finetuning of BM25 in order to get an optimal hyperparameters

In [3]:
# !head -n 1 /home/ryparmar/pyserini/en-latest/data/docs00.json

In [5]:
!python /home/ryparmar/pyserini/src/generate_subset.py \
    --dataset_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
    --subset_file /home/ryparmar/pyserini/en-latest/data/enwiki-subset.jsonl \
    --length 10000

#     --dataset_file /home/ryparmar/pyserini/en-latest/data/docs00.json \

Done!


In [6]:
# Generate queries and qrels files for the dev split
!python /home/ryparmar/pyserini/src/generate_queries_and_qrels.py \
    --dataset_file /home/ryparmar/pyserini/en-latest/data/enwiki-subset.jsonl \
    --output_queries_file /home/ryparmar/pyserini/en-latest/queries.par.subset.tsv \
    --output_qrels_file /home/ryparmar/pyserini/en-latest/qrels.par.subset.tsv \
    --granularity paragraph

Generating qrels...
Generating queries...
Done!


In [11]:
!cat /home/ryparmar/pyserini/en-latest/runs/finetuning/run.fever.bm25.k1_0.6.b_0.5.txt

In [7]:
!python /home/ryparmar/pyserini/src/tune_bm25.py \
    --runs_folder /home/ryparmar/pyserini/en-latest/runs/finetuning \
    --index_folder /home/ryparmar/pyserini/en-latest/anserini_index \
    --queries_file /home/ryparmar/pyserini/en-latest/queries.par.subset.tsv \
    --qrels_file /home/ryparmar/pyserini/en-latest/qrels.par.subset.tsv \
    --truth_file /home/ryparmar/pyserini/en-latest/data/enwiki-subset.jsonl

Done!


In [1]:
# !python /home/ryparmar/pyserini/src/tune_bm25.py \
#     --runs_folder /home/ryparmar/pyserini/en-latest/runs/finetuning \
#     --index_folder /home/ryparmar/pyserini/en-latest/indexes-full \
#     --queries_file /home/ryparmar/pyserini/en-latest/queries.par.subset.tsv \
#     --qrels_file /home/ryparmar/pyserini/en-latest/qrels.par.subset.tsv
# #     --truth_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl

### Perform Retrieval on the Dev

In [33]:
# Generate queries and qrels files for the dev split
!python /home/ryparmar/pyserini/src/generate_queries_and_qrels.py \
--dataset_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--output_queries_file /home/ryparmar/pyserini/en-latest/queries.par.dev.tsv \
--output_qrels_file /home/ryparmar/pyserini/en-latest/qrels.par.dev.tsv \
--granularity paragraph

Generating qrels...
Generating queries...
Done!


In [34]:
# claim_id, claim
!head -n 3 /home/ryparmar/pyserini/queries.par.dev.tsv

91198	Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.
194462	Tilda Swinton is a vegan.
137334	Fox 2000 Pictures released the film Soul Food.


In [35]:
# claim_id, doc_id (where the evidence is present), evidence?? in the code is 2 (directly number)
!head -n 3 /home/ryparmar/pyserini/qrels.par.dev.tsv

137334	0	Soul_Food_-LRB-film-RRB-	2
111897	0	Hispanic_and_Latino_Americans	2
111897	0	Telemundo	2


In [36]:
!head -n 3 /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl

{"id": 91198, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.", "evidence": [[[108548, null, null, null]]]}
{"id": 194462, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Tilda Swinton is a vegan.", "evidence": [[[227768, null, null, null]]]}
{"id": 137334, "verifiable": "VERIFIABLE", "label": "SUPPORTS", "claim": "Fox 2000 Pictures released the film Soul Food.", "evidence": [[[289914, 283015, "Soul_Food_-LRB-film-RRB-", 0]], [[291259, 284217, "Soul_Food_-LRB-film-RRB-", 0]], [[293412, 285960, "Soul_Food_-LRB-film-RRB-", 0]], [[337212, 322620, "Soul_Food_-LRB-film-RRB-", 0]], [[337214, 322622, "Soul_Food_-LRB-film-RRB-", 0]]]}


In [44]:
# Retrieval run
# k1=0.6 n=0.5 BM25 parameters found by finetuning
!python /home/ryparmar/pyserini/src/retrieve.py \
--hits 1000 --threads 1 \
--index /home/ryparmar/pyserini/en-latest/indexes-full \
--queries /home/ryparmar/pyserini/en-latest/queries.par.dev.tsv \
--output /home/ryparmar/pyserini/en-latest/run.fever-par.dev.tsv \
--k1 0.6 --b 0.5

Initializing BM25, setting k1=0.82 and b=0.68
Retrieving query 0 (15.468 s/query)
Retrieving query 100 (0.456 s/query)
Retrieving query 200 (0.279 s/query)
Retrieving query 300 (0.217 s/query)
Retrieving query 400 (0.186 s/query)
Retrieving query 500 (0.167 s/query)
Retrieving query 600 (0.153 s/query)
Retrieving query 700 (0.144 s/query)
Retrieving query 800 (0.137 s/query)
Retrieving query 900 (0.131 s/query)
Retrieving query 1000 (0.127 s/query)
Retrieving query 1100 (0.123 s/query)
Retrieving query 1200 (0.119 s/query)
Retrieving query 1300 (0.116 s/query)
Retrieving query 1400 (0.114 s/query)
Retrieving query 1500 (0.112 s/query)
Retrieving query 1600 (0.111 s/query)
Retrieving query 1700 (0.109 s/query)
Retrieving query 1800 (0.108 s/query)
Retrieving query 1900 (0.107 s/query)
Retrieving query 2000 (0.106 s/query)
Retrieving query 2100 (0.105 s/query)
Retrieving query 2200 (0.104 s/query)
Retrieving query 2300 (0.103 s/query)
Retrieving query 2400 (0.102 s/query)
Retrieving quer

In [39]:
# claim_id, predicted_document_id, rank  -- see that there up to 1000 hits (hits arg)
!head -n 1005 /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv | tail -n 10

91198	Patrick_McGuirk	996
91198	Ron_Adam	997
91198	Jullian_Taylor	998
91198	Richard_Alston_-LRB-_gridiron_football_-RRB-	999
91198	Andy_Puplis	1000
194462	Honor_Swinton_Byrne	1
194462	List_of_awards_and_nominations_received_by_Tilda_Swinton	2
194462	The_Souvenir_Part_II	3
194462	John_Swinton_of_Kimmerghame	4
194462	Julia_-LRB-_2008_film_-RRB-	5


In [40]:
!wc -l /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl

19998 /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl


In [41]:
!wc -l /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv

19991270 /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv


### Evaluation

In [19]:
import jsonlines
# Convert .tsv (anserini) prediction into .jsonl (drqa) predictions
def convert_to_drqa_format(inputfile, outputfile, truthfile):
    out = {}
    with jsonlines.open(truthfile) as ft:
        for line in ft.iter():
            out[str(line['id'])] = {'id': line['id'],
                               'label': line['label'],
                               'evidence': line['evidence']}
    with open(inputfile) as fr:
        for line in fr.readlines():
            claim_id, pred_docid, _ = line.split('\t')
            if claim_id in out and 'predicted_pages' in out[claim_id]:
                out[claim_id]['predicted_pages'].append(pred_docid)
            else:
                out[claim_id]['predicted_pages'] = [pred_docid]

        with jsonlines.open(outputfile, 'w') as fw:
            for key in out:
                fw.write(out[key])
                
# Convert .jsonl (drqa) predictions into .tsv (anserini) prediction 
def convert_to_anserini_format(inputfile, outputfile):
    out = []
    with jsonlines.open(inputfile) as ft:
        for line in ft.iter():
            for rank, docid in enumerate(line['predicted_pages']):
                out.append("{}\t{}\t{}\n".format(line['id'], docid, rank+1))

        with open(outputfile, 'w') as fw:
            for line in out:
                fw.write(line)

In [20]:
ls /home/ryparmar/pyserini/fever/

__MACOSX/  license.html          queries.par.dev.tsv
data/      paper_dev.jsonl       run.fever-par-finetuned-paper.dev.tsv
fever.db   paper_dev_pred.jsonl  train.jsonl
indexes/   qrels.par.dev.tsv     wiki-pages/


In [38]:
convert_to_drqa_format('/home/ryparmar/pyserini/en-latest/run.fever-par.dev.tsv',
                       '/home/ryparmar/pyserini/en-latest/run.fever-par.dev.jsonl',
                       '/mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl')

In [21]:
convert_to_anserini_format('/home/ryparmar/pyserini/fever/paper_dev_pred.jsonl',
                           '/home/ryparmar/pyserini/fever/paper_dev_drqa_k500.tsv')

In [14]:
!head -n 1 /mnt/data/factcheck/fever/data-en-latest/predictions/dev_drqa_k500.jsonl

{"id": 91198, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.", "evidence": [[[108548, null, null, null]]], "predicted_pages": ["Colin_Kaepernick", "2014_San_Francisco_49ers_season", "Alex_Smith", "The_Tip_-LRB-American_football-RRB-", "San_Francisco_Gold_Rush", "U.S._national_anthem_protests_-LRB-2016\u2013present-RRB-", "Steve_Collins_-LRB-American_football-RRB-", "2008_Humanitarian_Bowl", "Rob_Henry_-LRB-American_football-RRB-", "List_of_Kansas_City_Chiefs_starting_quarterbacks", "Tyrod_Taylor", "List_of_New_England_Patriots_starting_quarterbacks", "2016_New_England_Patriots_season", "Tommy_Maddox", "1990_New_York_Giants_season", "Peyton_Manning", "Super_Bowl_LIII_halftime_show", "1981_San_Francisco_49ers_season", "Jim_Plunkett", "Jeff_Garcia", "2008_Nevada_Wolf_Pack_football_team", "Tom_Brady", "Speak_for_Yourself_-LRB-talk_show-RRB-", "List_of_Indianap

In [25]:
!head -n 3 /home/ryparmar/pyserini/dev_drqa_k500.tsv

91198	Colin_Kaepernick	1
91198	2014_San_Francisco_49ers_season	2
91198	Alex_Smith	3


In [42]:
!head -n 3 /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl

{"id": 91198, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.", "evidence": [[[108548, null, null, null]]]}
{"id": 194462, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Tilda Swinton is a vegan.", "evidence": [[[227768, null, null, null]]]}
{"id": 137334, "verifiable": "VERIFIABLE", "label": "SUPPORTS", "claim": "Fox 2000 Pictures released the film Soul Food.", "evidence": [[[289914, 283015, "Soul_Food_-LRB-film-RRB-", 0]], [[291259, 284217, "Soul_Food_-LRB-film-RRB-", 0]], [[293412, 285960, "Soul_Food_-LRB-film-RRB-", 0]], [[337212, 322620, "Soul_Food_-LRB-film-RRB-", 0]], [[337214, 322622, "Soul_Food_-LRB-film-RRB-", 0]]]}


In [11]:
!head -n 3 /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv

91198	Colin_Kaepernick	1
91198	Alex_Smith	2
91198	U.S._national_anthem_protests_-LRB-_2016–present_-RRB-	3


In [39]:
!head -n 1 /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.jsonl

{"id": 91198, "label": "NOT ENOUGH INFO", "evidence": [[[108548, null, null, null]]], "predicted_pages": ["Colin_Kaepernick", "Alex_Smith", "U.S._national_anthem_protests_-LRB-_2016–present_-RRB-", "The_Tip_-LRB-_American_football_-RRB-", "San_Francisco_Gold_Rush", "2014_San_Francisco_49ers_season", "2017_San_Francisco_49ers_season", "Jim_Plunkett", "List_of_San_Francisco_49ers_starting_quarterbacks", "Tyrod_Taylor", "2007_San_Francisco_49ers_season", "Jimmy_Garoppolo", "1976_San_Francisco_49ers_season", "Elvis_Grbac", "2010_San_Francisco_49ers_season", "1981_San_Francisco_49ers_season", "Ed_Blount", "Lamar_Jackson", "1991_San_Francisco_49ers_season", "1988_San_Francisco_49ers_season", "Trent_Dilfer", "Jeff_Garcia", "1995_Pittsburgh_Steelers_season", "Super_Bowl_XIX", "Bruce_Threadgill", "Colt_McCoy", "1950_San_Francisco_49ers_season", "2012_St._Louis_Rams_season", "1970_San_Francisco_49ers_season", "Super_Bowl_XXIX", "List_of_Georgia_Tech_Yellow_Jackets_starting_quarterbacks", "List_o

In [16]:
# Evaluate DRQA on FEVER paper dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /home/ryparmar/pyserini/fever/paper_dev.jsonl \
--run_file /home/ryparmar/pyserini/fever/paper_dev_drqa_k500.tsv

k	Fully Supported	Oracle Accuracy
1	0.2532	0.5022
5	0.5536	0.7024
10	0.6584	0.7723
25	0.7595	0.8397
50	0.8242	0.8828
100	0.8663	0.9109
500	0.9290	0.9527


In [30]:
# Evaluate anserini finetuned on FEVER paper dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /home/ryparmar/pyserini/fever/paper_dev.jsonl \
--run_file /home/ryparmar/pyserini/fever/run.fever-par-finetuned-paper.dev.tsv

k	Fully Supported	Oracle Accuracy
1	0.3857	0.5905
5	0.6367	0.7578
10	0.7193	0.8129
25	0.8003	0.8669
50	0.8473	0.8982
100	0.8804	0.9203
500	0.9320	0.9547


In [25]:
# Evaluate anserini on FEVER dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--run_file /home/ryparmar/pyserini/en-latest/run.fever-par.dev.tsv

k	Fully Supported	Oracle Accuracy
1	0.2033	0.4689
5	0.3746	0.5831
10	0.4471	0.6314
25	0.5338	0.6892
50	0.5878	0.7252
100	0.6294	0.7529
500	0.6898	0.7932


In [52]:
# Evaluate anserini finetuned on FEVER dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--run_file /home/ryparmar/pyserini/en-latest/run.fever-par-0.6-0.5.dev.tsv

k	Fully Supported	Oracle Accuracy
1	0.2524	0.5016
5	0.4389	0.6260
10	0.5032	0.6688
25	0.5794	0.7196
50	0.6182	0.7455
100	0.6534	0.7689
500	0.7006	0.8004


In [53]:
# Evaluate drqa on FEVER dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--run_file /home/ryparmar/pyserini/en-latest/dev_drqa_k500.tsv

k	Fully Supported	Oracle Accuracy
1	0.2825	0.5217
5	0.5386	0.6924
10	0.6380	0.7587
25	0.7314	0.8209
50	0.7869	0.8579
100	0.8298	0.8865
500	0.8875	0.9250


# Evaluate using our evaluation script

In [60]:
!python /home/ryparmar/drchajan/src/evaluate_document_retrieval.py \
--actual_jsonl /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--pred_jsonl /mnt/data/factcheck/fever/data-en-latest/predictions/dev_drqa_k500.jsonl \
--max_evidence 500

[INFO] 2020-09-10 12:39:27,952 - LogHelper - Log Helper set up
[INFO] 2020-09-10 12:39:31,872 - EVALDR - Scores(precision=0.002055298647822243, recall=0.8874887488748875, f1=0.004101099727663442)


In [59]:
!python /home/ryparmar/drchajan/src/evaluate_document_retrieval.py \
--actual_jsonl /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--pred_jsonl /home/ryparmar/pyserini/en-latest/run.fever-par-0.6-0.5.dev.jsonl \
--max_evidence 500

[INFO] 2020-09-10 12:39:18,968 - LogHelper - Log Helper set up
[INFO] 2020-09-10 12:39:25,998 - EVALDR - Scores(precision=0.0016157452193816906, recall=0.7005700570057005, f1=0.0032240547070647844)


In [167]:
# Evaluate with trec_eval (TREC)
# Convert the runs and qrels to trec files
!python /home/ryparmar/pyserini/src/convert_msmarco_to_trec_run.py \
--input /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv \
--output /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.trec

!python /home/ryparmar/pyserini/src/convert_msmarco_to_trec_qrels.py \
--input /home/ryparmar/pyserini/qrels.par.dev.tsv \
--output /home/ryparmar/pyserini/qrels.par.dev.trec

Done!
Done!


In [169]:
# Run evaluation itself
!./home/ryparmar/pyserini/src/trec_eval.9.0.4/trec_eval -c -m all_trec \
/home/ryparmar/pyserini/qrels.par.dev.trec /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.trec

/usr/bin/sh: ./home/ryparmar/pyserini/src/trec_eval.9.0.4/trec_eval: No such file or directory
